# Tutorial 1: Coarse-grained protein-protein binding simulation 
Author: Jeremy Leung<br>
Email:&nbsp;&nbsp; jml230@pitt.edu

## Introduction

This tutorial involves running a coarse-grained protein-protein binding simulation using WESTPA's new binless resampler module. Users will also learn how to run dynamics through BioSimSpace.

## Learning Objectives


- Learn to customize the WESTPA's new binless resampler module.
- Run a protein-protein binding simulation using the "group by history" scheme.

## System Requirements

- AmberTools, numpy, matplotlib, ipympyl, mdtraj, and nglview are used in this tutorial. 
    - Install them using pip or conda install. It is recommended that you install all of them in a brand new conda environment for the purpose of this tutorial.
- mdtraj, nglview, matplotlib, ipympl, and numpy are optional for visualization purposes.
- AmberTools is necessary to run the simulation.

## Preparing the BnBs System

#### Preparing the protein structures

Because of the variability and lengthy steps required in preparing your Barnase-Barstar system, we will only outline the essential steps and resources on how to do that. A sample set of prepared structures are provided as part of this tutorial. Make sure to check your system at regular intervals as you proceed.

1. Download 1BRS from the [Protein Data Bank](https://www.rcsb.org/structure/1BRS).
2. Extract and refine the structure of one of the Barnase-Barstar pairs. Chains C and F are chosen for this tutorial. (e.g. using [Molprobity](http://molprobity.biochem.duke.edu/))

Minimize, re-solvate and equilibrate the system after each of the following steps, if possible. Restrain the protein backbone during the process.

3. Fill in the missing atoms with your program of choice. (e.g. using [Chimera](https://www.cgl.ucsf.edu/chimera/))
4. Coarse-grain and prepare your model following the instructions of the [SIRAH Forcefield](https://www.rcsb.org/structure/1BRS). We suggest following [Tutorial 5](http://www.sirahff.com/2012/08/sirah-forcefield-in-amber.html).
5. Separate your proteins to 15Å apart and generate multiple structures that are similarly solvated with the same amount of atoms. Example code listed here.

#### Visualization of the Barnase-Barstar System

We will now take a look at how one of the systems looks like after preparation. The water box is omitted for visibility. Barnase is red and Barstar is blue in the representation.

In [1]:
import mdtraj
import nglview
system = mdtraj.load('common_files/1brs_cg_solv.inpcrd', top='common_files/1brs_cg_solv.prmtop')
Bn = list(range(0,548)) # Barnase consists of beads #0 to 547, residues #1-110
Bs = list(range(548,991)) #Barstar consists of beads # 548 to 991, residues # 111-199
proteins = Bn + Bs # Bn + Bs
system = system.atom_slice(proteins) # Removing Water and Ions, just the proteins
view = nglview.show_mdtraj(system)
view.representations = [
    {"type": "ball+stick", "params": {
        "sele": "0-109", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": "110-198", "color": "blue"
    }}
]
view.background = ''
view

NGLWidget(background='')

The barnase-barstar simulation in this tutorial involves starting a weighted ensemble simulation with multiple basis states. The starting structures (after recycling or in iteration 1) are picked from the probabilities listed in 'bstates/bstates.txt'.

In [2]:
import numpy
import mdtraj
import nglview
lst = numpy.loadtxt('bstates/bstates.txt', usecols=2, dtype=str) # Reading basis state names
tpg = 'common_files/basis_all.prmtop' # Topology File for basis state (Shared between all bstates)

# Reading reference and setup
com = [] # list containing all CoM
a = mdtraj.load('reference/1brs_cg_eq2_resolv.pdb')
a_slice = a.atom_slice(list(range(0,548))) # Just barnase
com.append(numpy.squeeze(mdtraj.compute_center_of_mass(a_slice)))
a_slice = a.atom_slice(list(range(0,991))) # Both barnase + barstar

# Loading and superposing, storing Center of Mass (CoM) to list for heatmap
c = a_slice
for i in lst:
    b = mdtraj.load('bstates/' + i, top=tpg)
    b = b.atom_slice(list(range(0,991)))
    b.superpose(a_slice,atom_indices=list(range(0,548)))
    c = mdtraj.join([c,b], check_topology=False)
    # Just saving the CoM of barstar, since barnase is superimposed
    com.append(numpy.squeeze(mdtraj.compute_center_of_mass(b.atom_slice(list(range(548,991))))))
com = numpy.asarray(com)
print('Done')

Done


In [3]:
# Looking at all bstates frame-by-frame, assuming you ran the previous cell
view2 = nglview.show_mdtraj(c)
view2.representations = [
    {"type": "ball+stick", "params": {
        "sele": "0-109", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": "110-198", "color": "blue"
    }}
]
view2.center('0-109')
view2.control.zoom(-1.25)
view2

NGLWidget(max_frame=20)

In [4]:
# Looking at the coverage of the bstates, assuming you ran the previous 2 cells
# Comment out the following line or install ipypml if you have trouble viewing.
%matplotlib widget 
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import numpy

fig = matplotlib.pyplot.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

img1 = ax.scatter(com[0,0], com[0,1], com[0,2], s=20, marker='s', color='Red', label='Barnase')
img2 = ax.scatter(com[1:,0], com[1:,1], com[1:,2], s=50, marker='.', color='Blue', label='Barstar')

# Labels and Titles
ax.set_title("Basis Structures Coverage")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.legend()

show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Setting Up the Simulation

Utilizing the binless module is very similar to that of the default split-merge scheme. It only requires modifying `west.cfg` and attaching a `group.py` python file to dictate how trajectories are grouped.

For this tutorial, the `_find_parent_n()` and `walkers_by_history()` methods in `group.py` are used to group the trajectories. 

In [8]:
# `_find_parent_n()` traces each segment's parent n iterations back and passes it on as an array: 
with open('group.py') as f:
    file = f.readlines()
    for i in range(30,60):
        print(file[i])

def _find_parent_n(segment, n):

    iteration = (segment.n_iter - 1)

    parent_id = segment.parent_id

    try:

        if (len(segment.id_hist) < n):

            segment.id_hist = [(parent_id, iteration)] + segment.id_hist

        else:

            segment.id_hist.reverse()

            segment.id_hist.append((segment.parent_id, (segment.n_iter - 1)))

            segment.id_hist.reverse()

            del segment.id_hist[n:]

        parent_id = segment.id_hist[(len(segment.id_hist) - 1)][0]

        iteration = segment.id_hist[(len(segment.id_hist) - 1)][1]

    except AttributeError:

        data_manager = westpa.rc.get_data_manager()

        i = 0

        while (i < n) and parent_id >= 0:

            seg_id = parent_id

            iter_group = data_manager.get_iter_group(iteration)

            seg_index = iter_group['seg_index']

            parent_id = seg_index[seg_id]['parent_id']

            try:

                segment.id_hist.append((parent_id, iteration))

  

In [9]:
# `walkers_by_history()` utilizes that data previously generated by `_find_parent_n()` 
# to create groups with segments sharing parents. 
with open('group.py') as f:
    file = f.readlines()
    for i in range(7,28):
        print(file[i])

def walkers_by_history(we_driver, ibin, hist_length=25, **kwargs):

    '''Groups walkers inside of a bin according to their history.



    Creates a group, which takes the same data format as a bin, and then passes into the

    normal split/merge functions.'''

    # Pass in the bin object instead of the index

    log.debug('using group.walkers_by_history')

    log.debug('history length: {!r}'.format(hist_length))

    bin = we_driver.next_iter_binning[ibin]

    groups = dict()

    z = 0

    for segment in bin:

        if segment.n_iter > 1:

            ##par_iter = we_driver._find_parent_n(segment, hist_length)

            par_iter = _find_parent_n(segment, hist_length)

        else:

            par_iter = (0, 1)

        try:

            groups[par_iter].add(segment)

        except KeyError:

            groups[par_iter] = set([segment])



In [7]:
# The following options in `west.cfg` calls for the walkers_by_history function just before the 
# split and merging happens. `group_arguments` is a space for user-configurable options for 
# group functions. `hist_length` dictates how many iterations the function should look back.
with open('west.cfg') as f:
    file = f.readlines()
    for i in range(24,29):
        print(file[i])

  drivers:

    we_drivers: default

    group_function: group.walkers_by_history

    group_arguments:

      hist_length: 25



## Running the Simulation

The files to initialize and run the WESTPA simulation are included with this jupyter notebook. The simulation will take a while. Sample completed files for analysis are provided.

## Analyzing the Simulation

### See examples above!